In [36]:
import pandas as pd

In [37]:
ICD10_MAP = {
    'C50': 'Breast',
    'C51': 'Vulva',
    'C52': 'Vagina',
    'C53': 'Cervix Uteri',
    'C54': 'Corpus Uteri',
    'C55': 'Uterus',
    'C56': 'Ovary',
    'C57': 'Other and Unspecified Female Genital Organs',
    'C58': 'Placenta'
}

In [39]:
df = pd.read_csv('heisin_dag.csv')
LEVEL = 'level'
DIAG = 'diag_icd10'
FEMAL_CANCER_TYPE = 'cancer_type'
df[FEMAL_CANCER_TYPE] =df.apply(lambda x: ICD10_MAP.get(str(x[DIAG])[:3], None), axis=1)

primary_fc = df[(df[LEVEL] == 'Primary/main diagnosis') & (df[DIAG].str.contains('C5', na=False))]

In [40]:
primary_fc['has_cancer'] = True    # Mark presence of cancer
pivot_df = primary_fc.pivot_table(index='eid', 
                          columns=FEMAL_CANCER_TYPE, 
                          values='has_cancer', 
                          aggfunc='max', 
                          fill_value=False).astype(bool).reset_index()

# Display result
pivot_df

/tmp/ipykernel_247/602791804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_fc['has_cancer'] = True    # Mark presence of cancer


cancer_type,eid,Breast,Cervix Uteri,Corpus Uteri,Other and Unspecified Female Genital Organs,Ovary,Uterus,Vagina,Vulva
0,1000177,True,False,False,False,False,False,False,False
1,1000930,True,False,False,False,False,False,False,False
2,1001370,True,False,False,False,False,False,False,False
3,1001473,True,False,False,False,False,False,False,False
4,1001661,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
20120,6020450,True,False,False,False,False,False,False,False
20121,6020495,True,False,False,False,False,False,False,False
20122,6022005,True,False,False,False,False,False,False,False
20123,6022204,True,False,False,False,False,False,False,False


In [41]:
sum_of_true_values = pivot_df.drop(columns='eid').sum()
print("Sum of True values per cancer type:")
print(sum_of_true_values)

Sum of True values per cancer type:
cancer_type
Breast                                         18377
Cervix Uteri                                      39
Corpus Uteri                                     377
Other and Unspecified Female Genital Organs      188
Ovary                                           1814
Uterus                                            42
Vagina                                             8
Vulva                                             31
dtype: int64


In [42]:
pivot_df.to_csv('cleaned_diagnosis_data.csv')